In [1]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import os
import torch
import seaborn as sns
import copy

In [2]:
import sys
sys.path.append('code/BalancingControl')
import tmaze_utils as tu
import misc

Running on device cpu


In [3]:
training_file = os.path.join('raw_data', 'Training.csv')

In [4]:
training_csv = pd.read_csv(training_file, delimiter=';')
print(len(training_csv.columns))

181


In [5]:
# cutting out empty rows
training_csv = training_csv[1:18]

# note that animal 21 was excluded
training_csv = training_csv.drop(14)
print(training_csv.columns)

Index(['Rat Nr.', '1st', '2nd', '3rd', '4th', '5th', 'performance', '1st.1',
       '2nd.1', '3rd.1',
       ...
       '3rd.28', '4th.28', '5th.28', 'performance.28', '1st.29', '2nd.29',
       '3rd.29', '4th.29', '5th.29', 'performance.29'],
      dtype='object', length=181)


In [12]:
training_csv

,Rat Nr.,1st,2nd,3rd,4th,5th,performance,1st.1,2nd.1,3rd.1,...,3rd.28,4th.28,5th.28,performance.28,1st.29,2nd.29,3rd.29,4th.29,5th.29,performance.29
1,24.0,R,Lx,Lx,Lx,Lx,0.0,R,Lx,Lv,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,25.0,R,R,R,Lv,Lv,40.0,R,Lv,Lv,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,26.0,Lx,R,R,Lv,Lv,40.0,R,Lx,Lv,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,27.0,Lx,R,Lv,Lv,Lv,60.0,Lx,Lx,Lv,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,32.0,Lv,Lv,Lv,Lv,Lv,100.0,R,Lv,Lv,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,33.0,R,R,Lx,Lx,Lx,0.0,LX,LX,LX,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,34.0,Lv,R,R,Lv,Lv,60.0,R,LX,R,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,35.0,R,Lx,Lx,Lv,Lv,40.0,R,R,R,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,44.0,R,R,R,Lv,R,20.0,R,R,R,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,45.0,Lx,Lx,Lx,Lx,Lx,0.0,R,LX,LX,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# encoding:
# during training
# R = right -> wrong = no reward
# Lx = left, didnt consume reward -> no reward
# Lv = left, consumed reward -> reward
# during testing
# U = unbaited arm, wrong choice
# B = baited arm, correct choice

mapping = {"R": 0, "Lx": 1, "Lv": 1, "U": 1, "B": 0}

reward_mapping = {"R": 0, "Lx": 0, "Lv": 2, "B": 2, "U": 0}

# animals 24, 25, 26, 27, 32, 33, 34, 35 are alcohol-exposed, and animals 44, 45, 46, 47, 20, 21, 38, 39 are controls

animal_nr = list(training_csv['Rat Nr.'])

groups = {"control": animal_nr[:8], "cie": animal_nr[8:]}

print(groups)

n_animals = len(animal_nr)

{'control': [24.0, 25.0, 26.0, 27.0, 32.0, 33.0, 34.0, 35.0], 'cie': [44.0, 45.0, 46.0, 47.0, 20.0, 38.0, 39.0, nan]}


In [8]:
training_trials = 50
test_trials = 5

total_trials = 2*(training_trials+test_trials)

In [39]:
training_csv[training_csv["Rat Nr."]==animal_nr[2]].iterrows

def convert_letter_actions_to_numbers(df, k):

    actions = np.zeros((total_trials, 1)) - 1
    mask = np.zeros((total_trials,1)).astype(bool)
    observations = np.zeros((total_trials, 2))
    rewards = np.ones((total_trials, 2))

    count = 0
    for r in np.array(df[df['Rat Nr.'] == k])[0]:
        print(r)

convert_letter_actions_to_numbers(training_csv, 25.0)

25.0
R
R
R
Lv
Lv
40.0
R
Lv
Lv
Lv
Lv
80.0
nan
nan
nan
nan
nan
nan
Lx
Lv
Lv
Lv
Lv
80.0
U
B
U
U
U
20.0
nan
nan
nan
nan
nan
nan
Lv
Lv
Lv
Lx
Lx
60.0
nan
nan
nan
nan
nan
nan
LV
LV
LV
LV
LV
100.0
LV
LV
LV
LV
LV
100.0
LV
LV
LV
LV
LV
100.0
LV
LV
LV
LV
LV
100.0
LV
LV
LV
LV
LV
100.0
LV
LV
LV
LV
LV
100.0
LV
LV
LV
LV
LV
100.0
LV
LV
LV
LV
LV
100.0
LV
LV
LV
LV
LV
100.0
U
U
U
B
B
40.0
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


In [ ]:
def convert_letter_actions_to_numbers(df, k):

    actions = np.zeros((total_trials, 1)) - 1
    mask = np.zeros((total_trials,1)).astype(bool)
    observations = np.zeros((total_trials, 2))
    rewards = np.ones((total_trials, 2))

    count = 0
    for r in df[df['Rat Nr.'] == k].iterrows():
        letter = r[1][f'Unnamed: {1}'].strip()
        if letter == 'L' or letter == 'R':
            pass
        elif letter == 'B' or letter == 'U':
            if count < training_trials:
                count = training_trials
            else:
                count = 2*training_trials + test_trials
            pass
        else:
            print("Handle:", r[1][f'Unnamed: {1}'])
        for i in range(1, 6):
            # print(count, r[1][f'Unnamed: {i}'])
            print(r[1][f'Unnamed: {i}'])
            if r[1][f'Unnamed: {i}'] is np.nan:
                mask[count] = False
            else:
                letter_col = r[1][f'Unnamed: {i}'].strip()
                mask[count] = True
                actions[count] = mapping[letter_col]
                observations[count,1] = mapping[letter_col] + 1
                rewards[count,1] = reward_mapping[letter_col]
            count += 1

    return actions, mask, observations, rewards

In [10]:
def convert_letter_actions_to_numbers(df, k):

    actions = np.zeros((total_trials, 1)) - 1
    mask = np.zeros((total_trials,1)).astype(bool)
    observations = np.zeros((total_trials, 2))
    rewards = np.ones((total_trials, 2))

    count = 0
    for r in df[df['Unnamed: 0'] == str(k)].iterrows():
        letter = r[1][f'Unnamed: {1}'].strip()
        if letter == 'L' or letter == 'R':
            pass
        elif letter == 'B' or letter == 'U':
            if count < training_trials:
                count = training_trials
            else:
                count = 2*training_trials + test_trials
            pass
        else:
            print("Handle:", r[1][f'Unnamed: {1}'])
        for i in range(1, 6):
            # print(count, r[1][f'Unnamed: {i}'])
            print(r[1][f'Unnamed: {i}'])
            if r[1][f'Unnamed: {i}'] is np.nan:
                mask[count] = False
            else:
                letter_col = r[1][f'Unnamed: {i}'].strip()
                mask[count] = True
                actions[count] = mapping[letter_col]
                observations[count,1] = mapping[letter_col] + 1
                rewards[count,1] = reward_mapping[letter_col]
            count += 1

    return actions, mask, observations, rewards


def create_group_data(group_name, df):

    group_size = len(groups[group_name])
    data_list = []
    plot_dict = {"animal": np.zeros(total_trials*group_size), "correct": np.zeros(total_trials*group_size), "valid": np.zeros(total_trials*group_size).astype(bool), 
                 "group": [group_name]*total_trials*group_size, "trial": np.zeros(total_trials*group_size), "trial_type": []}

    for i,s in enumerate(groups[group_name]):
        print(s)
        actions, mask, observations, rewards = convert_letter_actions_to_numbers(df, s)
        data_list.append({"subject": torch.tensor([s]), "actions": torch.from_numpy(actions).long(), "observations": torch.from_numpy(observations).long(), 
                          "rewards":torch.from_numpy(rewards).long(), "states": torch.from_numpy(observations).long(), 'valid': torch.from_numpy(mask)})
        plot_dict["animal"][i*total_trials:(i+1)*total_trials] = s
        plot_dict["correct"][i*total_trials:(i+1)*total_trials] = rewards[:,1]//2
        plot_dict["valid"][i*total_trials:(i+1)*total_trials] = mask[...,0]
        plot_dict["trial"][i*total_trials:(i+1)*total_trials] = list(range(total_trials))
        plot_dict["trial_type"] += ["training"]*training_trials+["early test"]*test_trials+["training"]*training_trials+["late test"]*test_trials


    return data_list, plot_dict